In [72]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.options.display.max_colwidth = 500
from xgboost import XGBClassifier

In [73]:
dirs = ["HILTI_w2v_steps_walklength1_muse_2019_06_18_18_40_33_992813",
"HILTI_w2v_steps_walklength3_2019_06_18_18_57_07_825598",
"OAEI_emb_w2v_steps_walklength1_2019_06_18_18_07_24_762075",
"OAEI_emb_w2v_steps_walklength1_3grams_2019_06_18_19_25_11_020201",
"OAEI_emb_w2v_steps_walklength1_dim100_2019_06_18_21_27_54_025273",
"OAEI_emb_w2v_steps_walklength1_muse_2019_06_18_18_24_38_508409",
"OAEI_relw2v_steps_walklength1_2019_06_18_20_28_04_461710"]


In [74]:
basedir = "C:/Users/Alexander/DeepAnyMatch/result_data/hilti/"+dirs[2]+"/"

gs = pd.read_csv(basedir+"oaei_gold_standard5best.csv", encoding="UTF-8", sep="\t", header=None)
gs.columns = ['src_id','tgt_id','prediction']
#gs = gs.drop(['prediction'],axis=1)
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['tgt_id'], right_on=['label'])
gs.head()

,src_id,tgt_id,prediction,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_11,tgt_12,tgt_13,tgt_14,tgt_15,tgt_16,tgt_17,tgt_18,tgt_19,label_y
0,http://rdata2graph.sap.com/hilti_erp/install__channel_ms_f_124_d_1_1m#9e41112e-258c-4df2-a52c-ddb7582a2bc4,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702,0,0.864312,-2.393734,-3.486674,-0.733674,-2.871640,-0.088542,1.014541,...,0.288956,0.453579,1.14919,1.457903,2.676322,-1.791878,0.527996,2.608178,1.393705,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702
1,http://rdata2graph.sap.com/hilti_erp/install__channel_mq_41_2_12m#dce641e7-1a51-40de-ab47-8675c6f303f8,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702,0,0.220502,-2.957701,-3.114849,-1.048741,-3.015705,1.080840,1.618496,...,0.288956,0.453579,1.14919,1.457903,2.676322,-1.791878,0.527996,2.608178,1.393705,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702
2,http://rdata2graph.sap.com/hilti_erp/transversal_channel_msp_al_tc_4_5m#31b06da4-9100-4b63-9669-f9e877009fa7,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702,0,-1.088102,-1.990320,-3.998855,-1.170968,-3.702075,-0.476072,1.140320,...,0.288956,0.453579,1.14919,1.457903,2.676322,-1.791878,0.527996,2.608178,1.393705,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702
3,http://rdata2graph.sap.com/hilti_erp/install__channel_hac_70_175_2800_x_300_f#2ebf85bc-8d8a-43bb-adcb-8a3af3d492f7,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702,0,0.606353,-0.608742,-2.745347,1.906084,-2.228827,-2.343253,0.635373,...,0.288956,0.453579,1.14919,1.457903,2.676322,-1.791878,0.527996,2.608178,1.393705,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702
4,http://rdata2graph.sap.com/hilti_erp/install__channel_hac_50_2300_f#ec7dc61a-863d-4e80-aec2-26a7e796f70a,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702,0,0.511730,-2.159998,-4.021707,0.965570,-2.622225,-0.719871,0.056756,...,0.288956,0.453579,1.14919,1.457903,2.676322,-1.791878,0.527996,2.608178,1.393705,http://rdata2graph.sap.com/hilti_web/install__channel_spare_part#42b2c04a-cccd-45bf-800b-561cce901702


In [75]:
oaei_gold_standard3 = pd.read_csv(basedir+"final_trainset.csv_merged.csv",sep="\t",encoding="UTF-8")
oaei_gold_standard3 = oaei_gold_standard3[['src_id','tgt_id','label']]
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['tgt_id'], right_on=['label'])

oaei_gold_standard3 = extend_features(oaei_gold_standard3)
oaei_gold_standard3['syntactic_diff'] = oaei_gold_standard3.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3['plus_diff'] = oaei_gold_standard3.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3 = oaei_gold_standard3.drop(['label'], axis=1)
oaei_gold_standard3.rename({'label_x':'label'}, inplace=True, axis='columns')
oaei_gold_standard3 = oaei_gold_standard3.drop(['label_y'], axis=1)
oaei_gold_standard3.head()

.
.


,src_id,tgt_id,label,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_16,tgt_17,tgt_18,tgt_19,src_tgt_angle,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff
0,http://rdata2graph.sap.com/hilti_erp/data_cable_psa_92_usb_b#083ae2b7-e68e-4bda-92e8-5a50576fbf92,http://rdata2graph.sap.com/hilti_web/drive_cylinder_te2_a_02_assy_packed_spare_part#20cc13ca-0eb6-421c-afb1-ca1167b29a68,0,-1.143517,0.635042,-1.981195,-1.911831,-3.446341,0.124010,-0.686170,...,-0.951481,-0.398314,3.548356,1.560996,0.626684,7.695455,7.08046,8.286791,0.977273,0.770833
1,http://rdata2graph.sap.com/hilti_erp/filter_element_vc_20_40_hepa#aed178fc-1c3f-477e-adfb-1a36413de0ab,http://rdata2graph.sap.com/hilti_web/drive_cylinder_te2_a_02_assy_packed_spare_part#20cc13ca-0eb6-421c-afb1-ca1167b29a68,0,-2.093856,-0.647593,1.356329,1.041038,-3.223510,-1.820721,-1.405322,...,-0.951481,-0.398314,3.548356,1.560996,0.402316,7.016287,7.08046,6.322742,0.977273,0.708333
2,http://rdata2graph.sap.com/hilti_erp/gasket_set_te_1000___1500#835184b0-968d-4ae1-82e8-61eda1a25da4,http://rdata2graph.sap.com/hilti_web/drive_cylinder_te2_a_02_assy_packed_spare_part#20cc13ca-0eb6-421c-afb1-ca1167b29a68,0,-2.008078,-1.628333,-0.772681,-0.252711,-3.639766,-0.121282,-2.572310,...,-0.951481,-0.398314,3.548356,1.560996,0.239490,6.589254,7.08046,4.752688,0.977273,0.791667
3,http://rdata2graph.sap.com/hilti_erp/screw_6kt_m5x1_6p0_8x20#d8a2bd53-f062-4a37-b15e-5dfa9dc2101f,http://rdata2graph.sap.com/hilti_web/drive_cylinder_te2_a_02_assy_packed_spare_part#20cc13ca-0eb6-421c-afb1-ca1167b29a68,0,-1.932228,-2.436513,-1.443122,1.243557,-1.638522,-2.124756,-3.405989,...,-0.951481,-0.398314,3.548356,1.560996,0.275828,6.925987,7.08046,5.203518,1.000000,0.854167
4,http://rdata2graph.sap.com/hilti_erp/stator_te_1000_packed#001304a1-d365-4458-b4a9-440152fbc996,http://rdata2graph.sap.com/hilti_web/drive_cylinder_te2_a_02_assy_packed_spare_part#20cc13ca-0eb6-421c-afb1-ca1167b29a68,0,-0.743549,-2.040354,-1.017226,0.149583,-3.581116,0.003962,-3.090222,...,-0.951481,-0.398314,3.548356,1.560996,0.235576,7.171552,7.08046,4.892080,0.840909,0.708333


In [76]:
origindir = "C:/Users/Alexander/DeepAnyMatch/data/sap_hilti_data/balanced_walks/"
labels1 = dict()
categories1 = dict()
with open(origindir+"graph_triples_hilti_erp.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        if '<http://rdata2graph.sap.com/hilti_erp/property/mara_fert.maktx>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://rdata2graph.sap.com/hilti_erp/property/mara_fert.maktx ")
            if "maktx" not in line[0]:
                labels1[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories1[line[0]] = 'resource'
            else:
                categories1[line[0]] = line[1]
labels2 = dict()
categories2 = dict()
with open(origindir+"graph_triples_hilti_web.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        if '<http://rdata2graph.sap.com/hilti_web/property/products.name>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://rdata2graph.sap.com/hilti_web/property/products.name ")
            if ".name" not in line[0]:
                labels2[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories2[line[0]] = 'resource'
            else:
                categories2[line[0]] = line[1]

In [77]:
from sklearn.metrics.pairwise import *
def extend_features(df):
    src_pattern = "src_\d+"
    tgt_pattern = "tgt_\d+"
    src_dim = int(len([elem for elem in [re.match(src_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))
    tgt_dim = int(len([elem for elem in [re.match(tgt_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))


    def dotproduct(v1, v2):
        result = list()
        for i in range(len(v1)):
            result.append([np.dot(v1[i], v2[i])])
        return np.array(result)

    def length(v):
        return np.sqrt(dotproduct(v, v))

    def angle(v1, v2):
        return np.arctan(dotproduct(v1, v2) / (length(v1) * length(v2)))

    a = np.array(df[["src_" + str(i) for i in range(src_dim)]].values.tolist())
    b = np.array(df[["tgt_" + str(i) for i in range(tgt_dim)]].values.tolist())
    print(".")
    df['src_tgt_angle'] = paired_cosine_distances(a, b)
    print(".")
    #src_origin = np.full((len(df), src_dim), 0.0000001)
    #tgt_origin = np.full((len(df), tgt_dim), 0.0000001)
    #df['src_angle_to_origin'] = cosine_similarity(tgt_origin,a).diagonal()
    #print(".")
    #df['tgt_angle_to_origin'] = cosine_similarity(src_origin,b).diagonal()
    df['src_veclen'] = length(a)
    df['tgt_veclen'] = length(b)
    df['src_tgt_veclen'] = paired_euclidean_distances(a,b)#.diagonal()#length(a-b)
    df.head()
    
    df.fillna(0, inplace = True)
    return df


In [78]:
gs = extend_features(gs)
len(gs)

.
.


11519

In [79]:
memo = {}

def jacc(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    t = set([t[i:i+n] for i in range(len(t)-n+1)])
    s = set([s[i:i+n] for i in range(len(s)-n+1)])
    return 1-len([gram for gram in s if gram in t])/max(len(s), len(t))

def lev(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    return levenshtein(s,t)/max(len(s),len(t))
    
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1

    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])

    return res
gs['syntactic_diff'] = gs.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
gs['plus_diff'] = gs.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)

In [61]:
coefs = clf.coef_
for c in range(len(coefs[0])):
    print(cols[c] + " -> " + str(coefs[0,c]))

src_0 -> 0.14815128836565586
src_1 -> -0.28094213107564214
src_2 -> 0.25481541641849614
src_3 -> 0.43436988511677815
src_4 -> 0.03419341110073328
src_5 -> -0.4450602924290631
src_6 -> -0.14233047785651945
src_7 -> 0.07174942967261615
src_8 -> 0.1294440831191366
src_9 -> 0.07139694737373414
src_10 -> -0.0008181354249746885
src_11 -> -0.5181175126626455
src_12 -> -0.07065535927724143
src_13 -> 0.4987512413830652
src_14 -> 0.6420985769127596
src_15 -> -0.14726520202581395
src_16 -> 0.24071134475322573
src_17 -> 0.0034877779149912642
src_18 -> 0.09224728580933803
src_19 -> 0.08971551296885276
src_20 -> -0.23407270222610416
src_21 -> -0.09399661970121177
src_22 -> -0.19099817838074107
src_23 -> 0.18909520091364085
src_24 -> 0.19619922690473252
src_25 -> 0.10171097016741643
src_26 -> 0.13295390267584517
src_27 -> 0.13923718876120247
src_28 -> -0.14588650051554772
src_29 -> -0.14368656702220986
src_30 -> 0.24184953604564502
src_31 -> 0.04359965362887645
src_32 -> 0.040778906899045425
src_33 -

In [80]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
#oaei_gold_standard3 = pd.read_csv(basedir+"oaei_gold_standard2.csv_merged.csv",sep="\t",encoding="UTF-8")

cols = ['syntactic_diff']#[col for col in oaei_gold_standard3.columns if col not in ['label','Unnamed: 0','src_id','tgt_id','src_category','tgt_category','src_angle_to_origin','tgt_angle_to_origin']]#['src_tgt_angle', 'src_tgt_veclen', 'plus_diff', 'syntactic_diff']
X, y = oaei_gold_standard3[cols], oaei_gold_standard3.label
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
#XGBClassifier().fit(X, y)
#LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
        #random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.1,0:0.9}).fit(X, y)

X = gs[cols]
matchings = gs.loc[clf.predict(X)==1]
matchings.loc[:,'confidence'] = clf.predict_proba(matchings[cols])[:,1]
len(matchings)

c:\users\alexander\anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\alexander\anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


4476

In [63]:
#matchings = gs.loc[gs.plus_diff<0.25]

In [81]:
gs = matchings
len(matchings)

4476

In [65]:
len(set(gs.src_id))
from gensim.models import Doc2Vec, Word2Vec
model = Word2Vec.load(basedir+"w2v.model")
def get_training_material(nid):
            res = list()
            with open(basedir+"w2v_training_material.csv", mode="r", encoding="UTF-8") as f:
                for line in f:
                    if nodeid in line.split(" "):
                        for w in line.split(" "):
                            yield w

def mergedf(df1, df2):
            if df1 is None:
                return df2
            else:
                return df1.append(df2, ignore_index=True)


In [66]:
gs.loc[:,'total_score'] = 0

In [67]:
progress = 0
matchings = None
total = len(set(gs.src_id))
for nodeid in set(gs.src_id):#.union(gs.tgt_id)
                possible_matches_for_nodeid = gs.loc[(gs.src_id==nodeid) ]
        
                #possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid))))]



                progress += 1
                if len(possible_matches_for_nodeid)<1:
                    continue
                elif len(possible_matches_for_nodeid) == 1:
                    matching_pair = possible_matches_for_nodeid.head(1)
                    matchings = mergedf(matchings, matching_pair)
                    continue
                
                
                #print(str(progress), end="\r")
                print("         Computing rank-features: " + str(int(100*progress/total)) + "%.", end='\r')
                # In[312]:



                #model.docvecs.most_similar(0)


                # In[313]:


                #print('Closest in general:')
                #for val in model.docvecs.most_similar(i):
                #    try:
                #        print(documents_ids_A[int(val[0])])
                #    except:
                #        try:
                #            print(documents_ids_B[int(val[0])])
                #        except:
                #            print(str(val[0]) + " not found")


                # In[314]:


                ##print('Closest in terms of cosine similarity:')
                ##vecs = model.docvecs.doctag_syn0[np.array(get_possible_matches(nodeid))]
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = cosine_similarity(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=False)
                ##sorted_x.loc[:,'cos_score'] = 0
                ctr = 1
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x['cos_sim'] = sorted_x['cos_sim'].astype('float64')
                sorted_x = possible_matches_for_nodeid.sort_values(by=['src_tgt_angle'], ascending=True)
                sorted_x.loc[:,'cos_score'] = 0
                #maximum = sorted_x.head(1).src_tgt_angle.values[0]
                #sorted_x.loc[:,'diff_to_max'] = 1.0 - sorted_x.loc[:, 'src_tgt_angle'] / maximum
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['cos_sim']))
                    sorted_x.loc[index, 'cos_score'] = row['cos_score'] + 1/ctr
                    ctr += 1
                #sorted_x = possible_matches_for_nodeid.sort_values(by=['confidence'], ascending=False)
                sorted_x.loc[:,'conficence'] = 0 
                sorted_x.loc[:,'confidence_score'] = 0 
                ctr = 1
                for index, row in sorted_x.iterrows():
                    sorted_x.loc[index, 'confidence_score'] = row['confidence_score'] + 1/ctr
                    ctr += 1

                # In[315]:


                ##print('Closest in terms of Euclidean distance:')print('Closest in terms of Euclidean distance:')
                #sorted_x2 = sorted_x
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = euclidean_distances(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = sorted_x.sort_values(by=['src_tgt_veclen'], ascending=True)
                sorted_x.loc[:,'euclid_score'] = 0
                ctr = 1
                ##sorted_x.columns = ['euclid_sim' if col==0 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['euclid_sim']))
                    sorted_x.loc[index, 'euclid_score'] = row['euclid_score'] + 1/ctr
                    ctr += 1



                #print('Closest in terms of syntax:')
                sorted_x2 = sorted_x
                #vecs = model.wv[get_possible_matches(nodeid)]
                def edits(v1, v2s):
                    res = list()
                    v1 = v1.split("/")[-1]
                    for v2 in v2s:
                        v2 = v2.split("/")[-1]
                        res.append(editdistance.eval(v1, v2)/min(len(v1), len(v2)))
                    return np.array([res])
                #x = edits(nodeid, get_possible_matches(nodeid))
                #x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                #sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = possible_matches_for_nodeid.sort_values(by=['syntactic_diff'], ascending=True)
                sorted_x.loc[:,'syntax_score'] = 0
                ctr = 1
                sorted_x.columns = ['syntax_diff' if col==0 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['syntax_diff']))
                    sorted_x.loc[index, 'syntax_score'] = row['syntax_score'] + 1/ctr
                    ctr += 1



                sorted_x3 = sorted_x
                #o1 = get_training_material(nodeid)
                #inverted_dict = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    inverted_dict[tgt_id] = [tgt_id]
                #    for word in get_training_material(tgt_id):
                #        if word in inverted_dict.keys():
                #            inverted_dict[word].append(tgt_id)
                #        else:
                #            inverted_dict[word] = [tgt_id]

                #tgt_scores = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    tgt_scores[tgt_id]=0

                #for tupl in model.predict_output_word(o1, topn=99999999999999):
                #    if tupl[0] in inverted_dict.keys():
                #        for tgt_id in inverted_dict[tupl[0]]:
                #            tgt_scores[tgt_id] = tgt_scores[tgt_id] + tupl[1]
                sorted_x.loc[:,'probability'] = 0
                sorted_x.loc[:,'probability_score'] = 0
                #ctr=1
                #for item in sorted(tgt_scores.items(), key = lambda kv:(kv[1], kv[0]), reverse=True):
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability_score'] = 1/ctr
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability'] = float(item[1])
                #    ctr += 1

                # In[316]:


                #print('Closest in sum:')
                #sorted_x[['probability_score','probability']].merge(..., left_index=True, right_index=True)
                x = sorted_x[['probability_score','probability']].merge(sorted_x3['syntax_score'].to_frame().merge(sorted_x2, left_index=True, right_index=True), left_index=True, right_index=True)
                x.loc[:,'total_score'] = x['syntax_score'] + x['euclid_score'] + x['probability_score'] + x['confidence_score'] + x['cos_score']
                sorted_x = x.sort_values(by=['total_score'], ascending=False)
                ##sorted_x.columns = ['tgt_id' if col==1 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():#sorted_x.loc[sorted_x.total_score == max(sorted_x.total_score.values),:].iterrows():
                    matching_pair = pd.DataFrame([sorted_x.loc[index]])
                    matching_pair.loc[:,'src_id'] = nodeid
                    #print(nodeid + "\t" + row[1] + "\t" + str(row['total_score']) + "\t" + str(row['cos_score']) + "\t" + str(row['euclid_score']))
                    matchings = mergedf(matchings, matching_pair)
                

c:\users\alexander\anaconda3\envs\py36\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [83]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="PURESYTNAXML.csv"#"(ML)"+basedir.split("/")[-2]
#married_matches = pd.read_csv("C:/Users/D072202/RData2Graph/rdata2graph/data/sap_hilti_data/oaei_gold_standard1best.csv", sep="\t", encoding="UTF-8")
#married_matches.columns=['src_id','tgt_id','label']
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)



In [69]:
#matchings = matchings_saved
#matchings = gs

In [84]:
matchings_saved=matchings
matchings = matchings.sort_values(by=['syntactic_diff','src_tgt_veclen'], ascending=[True, True])
married_matchings = None
ctr = 0
while len(matchings) > 0:
                ctr += 1
                row = matchings.head(1)
                married_matchings = mergedf(married_matchings, pd.DataFrame(row))
                matchings = matchings.loc[~(matchings.src_id == row.src_id.values[0]) & ~(matchings.tgt_id == row.tgt_id.values[0])]
                print(str(len(matchings)) + " left     ", end="\r")

In [85]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="ONLYSYNTAX_Married"#"(MARRIED)"+basedir.split("/")[-2]
#married_matches = pd.read_csv(basedir + matchings_filename, sep="\t", encoding="UTF-8")
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in married_matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)

